In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from tqdm import tqdm


pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 15)

In [62]:
df_bal8_azr_v2 = pd.read_csv('C:\jupyter\SPP\inputoutput\df_bal8_azr_v2_calc.csv').drop('Unnamed: 0', axis=1)
well_phit_flag8 = df_bal8_azr_v2[df_bal8_azr_v2.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()
df_bal8_azr_v2_flag = df_bal8_azr_v2[df_bal8_azr_v2.well.isin(well_phit_flag8)]

In [ ]:
def fu_thickness_display(dataset):
    fig = go.Figure()
    for fm in [ 'Balakhany VIII sand', 'Balakhany VIII 25',
                'Balakhany VIII 20', 'Balakhany VIII 15', 'Balakhany VIII 10',
                'Balakhany VIII 5'][::-1]:
        if fm != 'Balakhany VIII':
            fig.add_trace(go.Bar(x=dataset[dataset.FORMATION == fm]['well'], 
                                y=dataset[dataset.FORMATION == fm]['TST'], name = fm))

    fig.update_layout(
        title='TST thickness of Balakhany FU by well',
        autosize=True,
        # width=1300,
        # height=600,
        xaxis_title='Well',
        yaxis_title='TST',
        margin=dict(l=10, r=10, t=50, b=10),
        barmode='stack',
        legend=dict(
                    x=0.9,
                    y=1,
                    traceorder='normal',
                    font=dict(
                        family='sans-serif',
                        size=12,
                        color='black'
                    ),
                    bgcolor='white',
                    bordercolor='Black',
                    borderwidth=1
                    )
    )
    fig.show()
    # fig.write_html(r'C:\jupyter\SPP\plots\bal8_fu_thicknes.html')

bal8 = df_bal8_azr_v2.groupby(['well','FORMATION_up'])['TST'].apply(lambda x: x.iloc[-1] - x.iloc[0]).reset_index()
bal8_fu = df_bal8_azr_v2.groupby(['well','FORMATION'])['TST'].apply(lambda x: x.iloc[-1] - x.iloc[0]).reset_index()
fu_thickness_display(bal8_fu)

In [ ]:
# checking well for bal8 5 formation
bal8_fu5_checklist = bal8_fu5_join[(bal8_fu5_join.TST_fu5>44) | (bal8_fu5_join.TST_fu5 < 11)].sort_values(by='TST_fu5').well.unique().tolist()
# for i in bal8_fu5_checklist:
#     print(i + ' &')

bal8_fu5_absence = []
for well in bal8.well.unique():
    if well not in bal8_fu5_join.well.unique():
        bal8_fu5_absence.append(well)

bal8_fu5_final = bal8_fu5_checklist + bal8_fu5_absence
bal8_fu5_final

In [162]:
def well_display_logs(dataset, wellname, fmname, net_var1, net_var2, ref_depth, depth_step, print_flag, path):
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    depth = data['TST']
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    net_init = data[net_var1]*100 
    net = data[net_var2]*100
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']
    calcite = data['calc']*100
    
    net_init_sum = (net_init.sum()*0.001).round(1)
    net_sum = (net.sum()*0.001).round(1)

    fig, ax = plt.subplots(1,4, figsize=(7,11), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[0].set_title(net_var1)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    twin0.vlines(0.5, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.5)
    twin0.grid(True)
    twin0.plot(net_init, depth, color='orange', linewidth=0.5, alpha=1)
    twin0.fill_betweenx(depth,net_init, color='orange', alpha=0.25)
    ax[1].plot(rhob, depth, color='red', alpha=0.75) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(True)
    ax[1].set_xlim(1.65, 2.65)
    ax[1].plot(calcite, depth, color='#4ebef1', linewidth=0.5, alpha=1)
    ax[1].fill_betweenx(depth,calcite, color='#4ebef1', alpha=0.5)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.set_xlim(0.6, 0)
    twin1.grid(False)
    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].tick_params(labelsize=10)
    ax[2].grid(True)
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    ax[2].set_title(net_var2)
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5, alpha=1)
    twin2.fill_betweenx(depth,net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    twin2.xaxis.set_ticks([])
    twin2.grid(False)
    twin3 = ax[2].twiny()
    twin3.plot(rdeep, depth, color='black', linewidth=1, alpha=1)
    twin3.set_xscale('log')
    twin3.set_xlim(1, 200)
    twin3.grid(False)
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + 
                 ' net_vsh: ' + str(net_init_sum) + 'm' 
                 ' net: ' + str(net_sum) + 'm', 
                 fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png', dpi=300)
    else:
        pass
    plt.show()
# for wellname in bal8_fu5_checklist[:1]:
    # well_display_logs(df_bal8_azr_v2, wellname, 'Balakhany VIII', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:/jupyter/SPP/output/')